In [1]:
import polars as pl
from op_datasets.processing.execute import reader, BatchInput
from op_datasets.logic.transforms.daily_address_summary import (
    daily_address_summary,
    CONDITIONS,
)
from op_coreutils.logger import structlog
from op_coreutils.bigquery.write import overwrite_partitions

from datetime import datetime
import os

os.environ["OPLABS_ENV"] = "prod"


log = structlog.get_logger()

2024-10-16 18:58:49 [info     ] Loaded env var: OP_ANALYTICS_VAULT


In [2]:
inputbatch: BatchInput = None
for _inputbatch in reader(
    chain="op", block_spec="105376883:+500", source_spec="goldsky"
):
    inputbatch = _inputbatch
    break

dataframes = inputbatch.dataframes

2024-10-16 18:58:50 [info     ] Loaded 5 items into vault.    
2024-10-16 18:58:51 [info     ] Initialized Clickhouse client.
2024-10-16 18:58:51 [info     ] Query success: blocks in 1.35s 2000 rows
2024-10-16 18:58:52 [info     ] Query success: transactions in 1.97s 14166 rows
2024-10-16 18:58:52 [info     ] Query success: logs in 2.40s 59381 rows


In [3]:
dataframes

{'blocks': shape: (2_000, 24)
 ┌───────┬─────────┬──────────┬────────────┬───┬──────────┬───────────┬────────────┬────────────────┐
 │ chain ┆ network ┆ chain_id ┆ dt         ┆ … ┆ gas_used ┆ gas_limit ┆ extra_data ┆ transaction_co │
 │ ---   ┆ ---     ┆ ---      ┆ ---        ┆   ┆ ---      ┆ ---       ┆ ---        ┆ unt            │
 │ str   ┆ str     ┆ i32      ┆ str        ┆   ┆ i64      ┆ i64       ┆ str        ┆ ---            │
 │       ┆         ┆          ┆            ┆   ┆          ┆           ┆            ┆ i64            │
 ╞═══════╪═════════╪══════════╪════════════╪═══╪══════════╪═══════════╪════════════╪════════════════╡
 │ op    ┆ mainnet ┆ 10       ┆ 2023-06-09 ┆ … ┆ 676868   ┆ 30000000  ┆ 0x         ┆ 5              │
 │ op    ┆ mainnet ┆ 10       ┆ 2023-06-09 ┆ … ┆ 1149120  ┆ 30000000  ┆ 0x         ┆ 7              │
 │ op    ┆ mainnet ┆ 10       ┆ 2023-06-09 ┆ … ┆ 3368746  ┆ 30000000  ┆ 0x         ┆ 9              │
 │ op    ┆ mainnet ┆ 10       ┆ 2023-06-09 ┆ … ┆ 417

In [4]:
# # check if our blocks are in the range we want
# log.info(f"Min block number: {dataframes["blocks"]["number"].min()}")
# log.info(f"Max block number: {dataframes["blocks"]["number"].max()}")

# log.info(f"Min block number: {datetime.fromtimestamp( dataframes["blocks"]["timestamp"].min() )}")
# log.info(f"Max block number: {datetime.fromtimestamp( dataframes["blocks"]["timestamp"].max() )}")

In [5]:
result = daily_address_summary(
    dataframes["transactions"],
    conditions=CONDITIONS,
    address="from_address",
    chain_id="chain_id",
    chain="chain",
    dt="dt",
)

In [6]:
# list(result.head()["hash"])
result.head()

from_address,chain_id,chain,dt,total_txs,total_txs_success,total_txs_fail,total_blocks,total_blocks_success,total_blocks_fail,min_block_number,max_block_number,block_interval_active,min_nonce,max_nonce,nonce_interval_active,total_l2_gas_used,total_l2_gas_used_success,total_l2_gas_used_fail,total_l1_gas_used,total_l1_gas_used_success,total_l1_gas_used_fail,total_gas_fees,total_gas_fees_success,total_gas_fees_fail,l2_contrib_gas_fees,l1_contrib_gas_fees,l1_blobgas_contrib_gas_fees,l1_l1gas_contrib_gas_fees,l2_contrib_gas_fees_base_fee,l2_contrib_gas_fees_priority_fee,avg_l2_gas_price_gwei,avg_l2_base_fee_gwei,avg_l2_priority_fee_gwei,avg_l1_gas_price_gwei,avg_l1_blob_base_fee_gwei,min_block_timestamp,max_block_timestamp,time_interval_active,unique_hours_active,perc_daily_hours_active,num_to_addresses,num_to_addresses_success,num_to_addresses_fail
str,i32,str,str,u32,u32,u32,u32,u32,u32,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,"decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,22]","decimal[*,17]","decimal[*,17]","decimal[*,17]","decimal[*,17]","decimal[*,17]",datetime[μs],datetime[μs],duration[μs],u32,f64,u32,u32,u32
"""0x6007c3e4883dcd8f9c375ca9dbe0…",10,"""op""","""2023-06-09""",23,23,0,23,23,0,105376049,105377980,1932,5834,5856,23,2467065,2467065,0,105656,105656,0,0.0013539667245995610000,0.0013539667245995610000,0.0000000000000000000000,0.0000000007545587550000,0.0013539659700408060000,0.0000000000000000000000,0.0013539659700408172827,0.0000000007545587550000,0.0000000000000000000000,0.00000030585280687,0.00000030585280687,0.00000000000000000,18.73516328784334065,0.00000000000000000,2023-06-09 22:47:55,2023-06-09 23:52:17,1h 4m 22s,2,0.083333,1,1,0
"""0xddb211cd80c3c4484638514d6aa1…",10,"""op""","""2023-06-09""",8,8,0,8,8,0,105376282,105376540,259,0,7,8,1788015,1788015,0,44748,44748,0,0.0005531253587214600000,0.0005531253587214600000,0.0000000000000000000000,0.0000000079504763970000,0.0005531174082450630000,0.0000000000000000000000,0.0005531174082450678614,0.0000000079504763970000,0.0000000000000000000000,0.00000444653786293,0.00000444653786293,0.00000000000000000,18.07122511944301421,0.00000000000000000,2023-06-09 22:55:41,2023-06-09 23:04:17,8m 36s,2,0.083333,5,5,0
"""0x2d6c0ce4958919e7c43c3a9f70ca…",10,"""op""","""2023-06-09""",4,4,0,4,4,0,105376330,105377966,1637,164,167,4,442130,442130,0,9776,9776,0,0.0001168805378722170000,0.0001168805378722170000,0.0000000000000000000000,0.0000000001591878220000,0.0001168803786843950000,0.0000000000000000000000,0.0001168803786843973537,0.0000000000245517120000,0.0000000001346361100000,0.00000036004754710,0.00000005553052722,0.00000030451701988,17.47931123308265139,0.00000000000000000,2023-06-09 22:57:17,2023-06-09 23:51:49,54m 32s,2,0.083333,4,4,0
"""0xa96614b5c2491121ac12cbab3f83…",10,"""op""","""2023-06-09""",1,1,0,1,1,0,105377827,105377827,1,70,70,1,96432,96432,0,2088,2088,0,0.0000235072283158480000,0.0000235072283158480000,0.0000000000000000000000,0.0000000000596914080000,0.0000235071686244400000,0.0000000000000000000000,0.0000235071686244409932,0.0000000000000000000000,0.0000000000596914080000,0.00000061900000000,0.00000000000000000,0.00000061900000000,16.45938965100000000,0.00000000000000000,2023-06-09 23:47:11,2023-06-09 23:47:11,0µs,1,0.041667,1,1,0
"""0x024cd06d84a2804d6077134db8cb…",10,"""op""","""2023-06-09""",1,1,0,1,1,0,105376233,105376233,1,387,387,1,118222,118222,0,3344,3344,0,0.0000388541499102090000,0.0000388541499102090000,0.0000000000000000000000,0.0000000000115857560000,0.0000388541383244530000,0.0000000000000000000000,0.0000388541383244530616,0.0000000000060293220000,0.0000000000055564340000,0.00000009800000000,0.00000005100000000,0.00000004700000000,16.98693056100000000,0.00000000000000000,2023-06-09 22:54:03,2023-06-09 22:54:03,0µs,1,0.041667,1,1,0


In [7]:
overwrite_partitions(result, "temp", "daily_address_summary", expiration_days=999)

2024-10-16 18:58:53 [info     ] Writing 1 partitions to BQ [2023-06-09 00:00:00 ... 2023-06-09 00:00:00]
2024-10-16 18:58:53 [info     ] DRYRUN OVERWRITE PARTITION: Wrote 2.9Krows 390.9KB to BQ temp.daily_address_summary
